# Decorators

Adapted from all over the internet

Decorators are a change of syntax. Being able to use them difeerentiates an intermediate pythonista from a noob one. It allows us to change the way functions, methods and classes behave.

## Example

Let's say you're defining methods on numbers:

In [1]:
def add(n1, n2):
    return n1 + n2

def multiply(n1, n2):
    return n1 * n2

def exponentiate(n1, n2):
    """Raise n1 to the power of n2"""
    import math
    return math.pow(n1, n2)

Python's dynamic typing will allow the two values to be anything accepted by python but the math.pow method will raise an error if it's not a number. But what if we want to ensure that it is a number before the error:

In [2]:
def is_number(n):
    """Return True iff n is a number."""
    # A number can always be converted to a float
    try:
        float(n)
        return True
    except ValueError:
        return False
    
def add(n1, n2):
    if not (is_number(n1) and is_number(n2)):
        print("Arguments must be numbers!")
        return
    return n1 + n2

def multiply(n1, n2):
    if not (is_number(n1) and is_number(n2)):
        print("Arguments must be numbers!")
        return
    return n1 * n2

def exponentiate(n1, n2):
    """Raise n1 to the power of n2"""
    if not (is_number(n1) and is_number(n2)):
        print("Arguments must be numbers!")
        return
    import math
    return math.pow(n1, n2)

By now you should know that this is wrong. Doing so much copy pasting and unnecesary definitions is not the python way.

We want the copy & pasted code to live in just one place, so any changes just go there (DRY code: Don't Repeat Yourself). So let's **refactor**. 

In [5]:
def validate_two_arguments(n1, n2):
    """
    Returns True if n1 and n2 are both numbers.
    """
    if not (is_number(n1) and is_number(n2)):
        return False
    return True

def add(n1, n2):
    if validate_two_arguments(n1, n2):
        return n1 + n2

def multiply(n1, n2):
    if validate_two_arguments(n1, n2):
        return n1 * n2

def exponentiate(n1, n2):
    """Raise n1 to the power of n2"""
    if validate_two_arguments(n1, n2):
        import math
        return math.pow(n1, n2)

This is definitely better. But there's still some repeated logic. Like, what if we want to return an error if we don't get numbers, or print something before running the code? We'd still have to make the changes in multiple places. The code isn't DRY. 

## Basic decorators

We can refactor further with the **decorator pattern**.

We want to write something that looks like
    
    @decorator
    def add(n1, n2):
        return n1 + n2

so that all the logic about validating `n1` and `n2` lives in one place, and the functions just do what we want them to do. 
        
Since the @ syntax just means `add = decorator(add)`, we know the decorator needs to take a function as an argument, and it needs to return a function. (This should be confusing at first. Functions returning functions are scary, but think about it until that doesn't seem outlandish to you.)

This returned function should act the same way as `add`, so it should take two arguments. And within this returned function, we want to first check that the arguments are numbers. If they are, we want to call the original function that we decorated (in this case, `add`). If not, we don't want to do anything. Here's what that looks like (there's a lot here, so use the comments to understand what's happening):

In [6]:
# The decorator: takes a function.
def validate_arguments(func):
    # The decorator will be returning wrapped_func, a function that has the 
    # same signature as add, multiply, etc.
    def wrapped_func(n1, n2):
        # If we don't have two numbers, we don't want to run the function. 
        # Best practice ("be explicit") is to raise an error here 
        # instead of just returning None.
        if not validate_two_arguments(n1, n2):
            raise Exception("Arguments must be numbers!")
        # We've passed our checks, so we can call the function with the passed in arguments.
        # If you like, think of this as
        #   result = func(n1, n2)
        #   return result
        # to distinguish it from the outer return where we're returning a function.
        return func(n1, n2)
    # This is where we return the function that has the same signature.
    return wrapped_func

In [7]:
@validate_arguments
def add(n1, n2):
    return n1 + n2
# Don't forget, the @ syntax just means
# add = validate_decorator(add)

print(add(1, 3))
try:
    add(2, 'hi')
except Exception as e:
    print("Caught Exception: {}".format(e))

4
Caught Exception: Arguments must be numbers!


This pattern is nice because we've even refactored out all the validation logic (even the "if blah then blah" part) into the decorator. 

## Generalizing with \*args and \**kwargs

What if we want to validate a function that has a different number of arguments?

In [9]:
@validate_arguments  # Won't work!
def add3(n1, n2, n3):
    return n1 + n2 + n3

add3(1, 2, 3)

TypeError: wrapped_func() takes 2 positional arguments but 3 were given

We can't decorate this because the wrapped function expects 2 arguments. 

Here's where we use the `*` symbol. I'll write out the code so you can see how it looks, and we'll look at what `*args` is doing below. 

In [10]:
# The decorator: takes a function.
def validate_arguments(func):
    # Note the *args! Think of this as representing "as many arguments as you want". 
    # So this function will take an arbitrary number of arguments.
    def wrapped_func(*args):
        # We just want to apply the check to each argument.
        for arg in args:
            if not is_number(arg):
                raise Exception("Arguments must be numbers!")
        # We also want to make sure there's at least two arguments.
        if len(args) < 2:
            raise Exception("Must specify at least 2 arguments!")
        
        # We've passed our checks, so we can call the function with the 
        # passed-in arguments.
        # Right now, args is a tuple of all the different arguments passed in 
        # (more explanation below), so we want to expand them back out when 
        # calling the function.
        return func(*args)
    return wrapped_func

In [11]:
@validate_arguments  # This works
def add3(n1, n2, n3):
    return n1 + n2 + n3

add3(1, 2, 3)

6

In [12]:
@validate_arguments  # And so does this
def addn(*args):
    """Add an arbitrary number of numbers together"""
    cumu = 0
    for arg in args:
        cumu += arg
    return cumu
print(addn(1, 2, 3, 4, 5))
# range(n) gives a list, so we expand the list into positional arguments...
print(addn(*range(10)))

15
45



Congrats, you now understand decorators! You can do tons of other stuff with them, but hopefully now you're equipped to read the other guides online. 



---
# Advanced decorators

This section will introduce some of the many other useful ways you can use decorators. We'll talk about
* Passing arguments into decorators
* `functools.wraps`
* Returning a different function
* Decorators and objects.



## Decorators with arguments

A common thing to want to do is to do some kind of configuration in a decorator. For example, let's say we want to define a `divide_n` method, and to make it easy to use we want to hide the existence of integer division. Let's define a decorator that converts arguments into floats.

In [ ]:
def convert_arguments(func):
    """
    Convert func arguments to floats.
    """
    # Introducing the leading underscore: (weakly) marks a private 
    # method/property that should not be accessed outside the defining
    # scope. Look up PEP 8 for more. 
    def _wrapped_func(*args):
        new_args = [float(arg) for arg in args]
        return func(*new_args)
    return _wrapped_func

@convert_arguments
@validate_arguments
def divide_n(*args):
    cumu = args[0]
    for arg in args[1:]:
        cumu = cumu / arg
    return cumu

In [ ]:
# The user doesn't need to think about integer division!
divide_n(103, 2, 8)

But now let's say we want to define a `divide_n_as_integers` function. We could write a new decorator, or we could alter our decorator so that we can specify what we want to convert the arguments to. Let's try the latter. 



In [ ]:
def convert_arguments_to(to_type=float):
    """
    Convert arguments to the given to_type by casting them.
    """
    def _wrapper(func):
        def _wrapped_func(*args):
            new_args = [to_type(arg) for arg in args]
            return func(*new_args)
        return _wrapped_func
    return _wrapper


@validate_arguments
def divide_n(*args):
    cumu = args[0]
    for arg in args[1:]:
        cumu = cumu / arg
    return cumu


@convert_arguments_to(to_type=int)
def divide_n_as_integers(*args):
    return int(divide_n(*args))


@convert_arguments_to(to_type=float)
def divide_n_as_float(*args):
    return divide_n(*args)


print(divide_n_as_float('6', 3))
print(divide_n_as_integers('6', 3))

Did you notice the tricky thing about creating a decorator that takes arguments? **We had to create a function to "return a decorator".** The outermost function, `convert_arguments_to`, returns a function that takes a function, which is what we've been calling a "decorator". 

To think about why this is necessary, let's start from the form that we wanted to write, and unpack from there. We wanted to be able to do:

    @decorator(decorator_arg)
    def myfunc(*func_args):
        pass
        
Unpacking the syntactic sugar gives us

    def myfunc(*func_args): 
       pass
    myfunc = decorator(decorator_arg)(myfunc)
      
Written this way, it should immediately be clear that `decorator(decorator_arg)` **returns a function that takes a function**. 

So that's how you write a decorator that takes an argument: it actually has to be a function that takes your decorator arguments, and returns a function that takes a function. 

## functools.wraps
If you've played around with the examples above, you might've seen that the name of the wrapped function changes after you apply a decorator... And perhaps more importantly, the docstring of the wrapped function changes too (this is important for when generating documentation, e.g. with Sphinx). 

In [ ]:
@validate_arguments
def foo(*args):
    """foo frobs bar"""
    pass


print(foo.__name__)
print(foo.__doc__)

[`functools.wraps`](https://docs.python.org/2/library/functools.html#functools.wraps) solves this problem. Use it as follows:

In [ ]:
from functools import wraps

def better_validate_arguments(func):
    @wraps(func)
    def wrapped_func(*args):
        for arg in args:
            if not is_number(arg):
                raise Exception("Arguments must be numbers!")
        if len(args) < 2:
            raise Exception("Must specify at least 2 arguments!")
        return func(*args)
    return wrapped_func


@better_validate_arguments
def bar(*args):
    """bar frobs foo"""
    pass


print(bar.__name__)
print(bar.__doc__)

Think of the `@wraps` decorator making it so that `wrapped_func` knows what function it originally wrapped. 

## Returning a different function

Decorators don't even have to return the function that's passed in. You can have some fun with this...

In [ ]:
def jedi_mind_trick(func):
    def _jedi_func():
        return "Not the droid you're looking for"
    return _jedi_func

    
@jedi_mind_trick
def get_droid():
    return "Found the droid!"


get_droid()

But more seriously, this can be useful for things like

* Authentication: you don't want to return the function if the user isn't recognized, instead redirecting to a login page (e.g. you could check an environment variable)
* Disabling test methods when deployed to a production environment


# Decorators by case

Functools have plenty of useful decorators that you should look into. But we will look at custom decorators according to their use cases

### Decorators for annotation
Decorators can add annotations to functions when they're declared. For instance, suppose we want to label two types of functions, "red" functions and "blue" functions: 

In [ ]:
def red(fn):
    fn.color = 'red'
    return fn

def blue(fn):
    fn.color = 'blue'
    return fn

@red
def combine(a, b):
    result = []
    result.extend(a)
    result.extend(b)
    return result

@blue
def unsafe_combine(a, b):
    a.extend(b)
    return a

@blue
def combine_and_save(a, b):
    result = a + b
    with open('combined', 'w') as f:
        f.write(repr(result))
    return result

In [ ]:
def combine_using(fn, a, b):
    if hasattr(fn, 'color') and fn.color == 'blue':
        print("Sorry, only red functions allowed here!")
        return combine(a, b)  # fall back to default implementation
    return fn(a, b)

a = [1, 2]
b = [3, 4]
print(combine_using(unsafe_combine, a, b))
a

the `red` and `blue` decorators offer the benefits of being:
- highly and immediately visible
- inherently closely attached to/above the function definition
- consistent and foolproof (no room for typos).

If you've ever used [`pytest`](https://docs.pytest.org/en/latest/), this is what `@pytest.mark.parametrize`, `@pytest.mark.skip`, `@pytest.mark.[etc]` are doing - simply setting attributes on your test function, (some of) which are later used by the framework to dictate how the test is to be run.

### Decorators for registration
Sometimes, we want to have a centralized means of discovering a number of different functions. Decorators are a convenient means of doing this.

In [ ]:
FUNCTION_REGISTRY = []

def registered(fn):
    FUNCTION_REGISTRY.append(fn)
    return fn

@registered
def step_1():
    print("Hello")
    
@registered
def step_2():
    print("world!")

In [ ]:
def run_all():
    for function in FUNCTION_REGISTRY:
        function()
        
run_all()

while we could have accomplished the same thing by doing:

In [ ]:
def step_1():
    print("Hello")
    
def step_2():
    print("world!")
    
FUNCTION_REGISTRY = [step_1, step_2]

This means that, looking at the definition of `step_1` in isolation, we don't _know_ that it's been registered and, further, if we want to figure out how or why it's being run, we need to first identify that it's included in `FUNCTION_REGISTRY`, and then see that `FUNCTION_REGISTRY` is used in `run_all`. Additionally, if we add a `step_3` - possibly referencing our other step functions - we have to remember to add it to `FUNCTION_REGISTRY` when we're done. This is harder to forget when you see `step_1` and `step_2` very visibly decorated with `@registered`.

### Decorators for verification
Discovering bugs at runtime is a big bummer. Even moreso when they pop up at the end of a very long-running script or program, or in code running in production. Because decorators are evaluated at function definition time, we can use them to give us "compile"-time assurances immediately when a module is imported.

For instance, pretty frequently you'll want to use other languages or DSLs within Python: regular expressions, SQL, XPath, etc. The problem is that these are almost always represented as strings, not code, meaning that you can't benefit from syntax checking (although it [doesn't have to be this way](https://github.com/hchasestevens/xpyth)). Using a decorator, we can at least be alerted when the strings in our function have mismatched brackets - regardless of if or when the function is run:

In [ ]:
def brackets_balanced(s):
    brackets = {
        opening: closing
        for opening, closing in 
        '() {} []'.split()
    }
    closing = set(brackets.values())
    stack = []
    for char in s:
        if char not in closing:
            if char in brackets:
                stack.append(brackets[char])
            continue
        try:
            expected = stack.pop()
        except IndexError:
            return False
        if char != expected:
            return False
    return not stack

def ensure_brackets_balanced(fn):
    for const in fn.__code__.co_consts:
        if not isinstance(const, str) or brackets_balanced(const):
            continue
        print(
            "WARNING - {.__name__} contains unbalanced brackets: {}".format(
                fn, const
            )
        )
    return fn

@ensure_brackets_balanced
def get_root_div_paragraphs(xml_element):
    return xml_element.xpath("//div[not(ancestor::div]/p")

### Decorators for dispatch

It's often very convenient to not explicitly, yourself, decide what functions should be run under what circumstances on which inputs, but to instead simply indicate the necessary circumstances for a function, and then let the computer decide what function to run by using that information. Decorators are a clean way of establishing these mappings between input conditions and handling strategy.

For instance, consider the `IntStore` example above. The `constructor` decorator _annotates_ each method, the `register_constructors` class decorator _registers_ each of these on the class, and the `from_auto` method uses this information to _dispatch_ on the input type. 

This can be taken even further by creating a suite of strategies, each with their own preconditions, available for use by the program at run time. Giving your program _options_ like this can give you a _robustness_ and _flexibility_ that is very desirable for some applications - for instance, web scraping, in which the document you're parsing can vary (on a single site) depending on a huge number of factors.

Anyway, here's an example:

In [ ]:
STRATEGIES = []

def precondition(cond):
    def decorator(fn):
        fn.precondition_met = lambda **kwargs: eval(cond, kwargs)
        STRATEGIES.append(fn)
        return fn
    return decorator

@precondition("s.startswith('The year is ')")
def parse_year_from_declaration(s):
    print("Using declaration")
    return int(s[-4:])

@precondition("any(substr.isdigit() for substr in s.split())")
def parse_year_from_word(s):
    print("Using words")
    for substr in s.split():
        try:
            return int(substr)
        except Exception:
            continue
            
@precondition("'-' in s")
def parse_year_from_iso(s):
    print("Using iso")
    from dateutil import parser
    return parser.parse(s).year

def parse_year(s):
    for strategy in STRATEGIES:
        if strategy.precondition_met(s=s):
            return strategy(s)
        
print(parse_year("It's 2017 bro."))

print(parse_year('The year is 2018'))
print(parse_year('01-01-2019'))